Librerías y Funciones propias:

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('..')) 

import numpy as np
import pyroomacoustics as pra
import matplotlib.pyplot as plt
from scipy.io import wavfile
from algoritmos.CC import timeAveragedcrossCorrelation
from algoritmos.GCC import generalizedCrossCorrelation
from algoritmos.DOA import doa
from helpers.addNoise import addNoise

Señales de entrada:

In [ ]:
fs, sourceSignal = wavfile.read("../audios/p262_001.wav")
sourceSignal = addNoise(sourceSignal, 30)

if sourceSignal.dtype != np.float32 and sourceSignal.dtype != np.float64:
    sourceSignal = sourceSignal.astype(np.float32)

if np.max(np.abs(sourceSignal)) > 1:
    sourceSignal = sourceSignal / np.max(np.abs(sourceSignal))


Simulación:

In [ ]:
# Room dimensions (width * length)
roomDimensions = [8, 8, 3]

# Create a room
rt60Target = 0.5
eAbsorption, max_order = pra.inverse_sabine(rt60Target, roomDimensions)
room = pra.ShoeBox(
    roomDimensions, fs=fs, materials=pra.Material(eAbsorption), max_order=max_order
)

# Sound source position (x,y,z)
sourcePosition = [4, 6.5, 1.5]

# Add the source to the room
room.add_source(sourcePosition, signal=sourceSignal)

# Define a linear Microphone Array: 4 mics in a row
distanceBetweenMics = 0.15
initialMicPosition = [3.775, 2.5, 1]

# Calculate positions for 4 microphones
micPositions = np.array([
    [initialMicPosition[0] + i * distanceBetweenMics, 
     initialMicPosition[1],
     initialMicPosition[2]]
    for i in range(4)
])

# Add microphones to the room
room.add_microphone_array(pra.MicrophoneArray(micPositions.T, room.fs))

# Run the room simulation
room.simulate()

# Get the microphone signals
micSignals = room.mic_array.signals
numberSamples = micSignals.shape[1]
timeAxis = np.linspace(0, numberSamples / room.fs, numberSamples)

rt60Real = room.measure_rt60()
print("The desired RT60 was {}".format(rt60Target))
print("The measured RT60 is {}".format(rt60Real[1, 0]))

Algoritmos

Time Averaged Cross Correlation

In [ ]:
delays = []
for i in range(3):
    delays.append((timeAveragedcrossCorrelation(micSignals[i], micSignals[i + 1], returnDelay=True, graphs=False)[1]) / fs)

print("Delays between microphones (in seconds):", delays)

print("DOA time averaged cross correlation: ", doa(delays, distanceBetweenMics))

GCC: Classic

In [ ]:
delays = []
for i in range(3):
    delays.append((generalizedCrossCorrelation(micSignals[i], micSignals[i + 1], mode='classic', returnDelay=True, graphs=True)[1]) / fs)

print("Delays between microphones (in seconds):", delays)

print("DOA classic cross correlation: ", doa(delays, distanceBetweenMics))

GCC: Roth

In [ ]:
delays = []
for i in range(3):
    delays.append((generalizedCrossCorrelation(micSignals[i], micSignals[i + 1], mode='roth', returnDelay=True, graphs=True)[1]) / fs)

print("Delays between microphones (in seconds):", delays)

print("DOA Roth: ", doa(delays, distanceBetweenMics))

GCC: SCOT

In [ ]:
delays = []
for i in range(3):
    delays.append((generalizedCrossCorrelation(micSignals[i], micSignals[i + 1], mode='scot', returnDelay=True, graphs=False)[1]) / fs)

print("Delays between microphones (in seconds):", delays)

print("DOA SCOT: ", doa(delays, distanceBetweenMics))

GCC: Phase

In [ ]:
delays = []
for i in range(3):
    delays.append((generalizedCrossCorrelation(micSignals[i], micSignals[i + 1], mode='phase', returnDelay=True, graphs=False)[1]) / fs)

print("Delays between microphones (in seconds):", delays)

print("DOA Phase: ", doa(delays, distanceBetweenMics))